In [32]:
import pandas as pd
import numpy as np
import folium
import geopandas as gpd
from shapely.geometry import Point, MultiPolygon, Polygon
from feature_engine.categorical_encoders import OneHotCategoricalEncoder

### Add the neighborhood information for the houses

In [43]:
#Load the data
houses = pd.read_csv('Data/houses.csv')
houses.head(2)

,latitude,longitude,home_size,lot_size,zoning,sex_offenders,crime_index,enviornmental_hazards,school_quality,bedrooms,bathrooms,date,sale_price,sale_price_cpi,age
0,34.190532,-118.145081,1199.0,7207.0,LCR175,5.0,Moderate,4.0,Poor,3.0,2.0,2020-11-01,1010000.0,13.823924,70.0
1,33.899958,-117.990815,1254.0,6033.0,LMR1,2.0,Low,3.0,Above Average,3.0,3.0,2020-11-01,685000.0,13.435637,63.0


In [44]:
hoods = gpd.read_file('Los Angeles Neighborhood Map.geojson')
hoods = hoods.rename(columns={'name': 'neighborhood'})
hoods['polygon'] = hoods['geometry'].apply(lambda x: Polygon(x[0]))
hoods.head(2)

,external_i,neighborhood,location,latitude,slug_1,sqmi,display_na,set,slug,longitude,name_1,kind,type,geometry,polygon
0,acton,Acton,POINT(34.497355239240846 -118.16981019229348),-118.16981019229348,None,39.3391089485,Acton L.A. County Neighborhood (Current),L.A. County Neighborhoods (Current),acton,34.497355239240846,None,L.A. County Neighborhood (Current),unincorporated-area,"MULTIPOLYGON (((-118.20262 34.53899, -118.1894...","POLYGON ((-118.20262 34.53899, -118.18947 34.5..."
1,adams-normandie,Adams-Normandie,POINT(34.031461499124156 -118.30020800000011),-118.30020800000011,None,0.805350187789,Adams-Normandie L.A. County Neighborhood (Curr...,L.A. County Neighborhoods (Current),adams-normandie,34.031461499124156,None,L.A. County Neighborhood (Current),segment-of-a-city,"MULTIPOLYGON (((-118.30901 34.03741, -118.3004...","POLYGON ((-118.30901 34.03741, -118.30041 34.0..."


In [45]:
#Turn the lat, long info into a geometry point
gdf = gpd.GeoDataFrame(houses, geometry=[Point(xy) for xy in zip(houses.longitude, houses.latitude)])
gdf.head(2)

,latitude,longitude,home_size,lot_size,zoning,sex_offenders,crime_index,enviornmental_hazards,school_quality,bedrooms,bathrooms,date,sale_price,sale_price_cpi,age,geometry
0,34.190532,-118.145081,1199.0,7207.0,LCR175,5.0,Moderate,4.0,Poor,3.0,2.0,2020-11-01,1010000.0,13.823924,70.0,POINT (-118.14508 34.19053)
1,33.899958,-117.990815,1254.0,6033.0,LMR1,2.0,Low,3.0,Above Average,3.0,3.0,2020-11-01,685000.0,13.435637,63.0,POINT (-117.99081 33.89996)


In [46]:
#This function will classify every house and label it an appropriate neighborhood
def find_neighborhood(coordinates):
    for index, hood in hoods.iterrows():
        if hood['polygon'].contains(coordinates):
            return hood['neighborhood']
        else:
            continue 

gdf['neighborhood'] = gdf['geometry'].apply(find_neighborhood)
gdf.head()

,latitude,longitude,home_size,lot_size,zoning,sex_offenders,crime_index,enviornmental_hazards,school_quality,bedrooms,bathrooms,date,sale_price,sale_price_cpi,age,geometry,neighborhood
0,34.190532,-118.145081,1199.0,7207.0,LCR175,5.0,Moderate,4.0,Poor,3.0,2.0,2020-11-01,1010000.0,13.823924,70.0,POINT (-118.14508 34.19053),Altadena
1,33.899958,-117.990815,1254.0,6033.0,LMR1,2.0,Low,3.0,Above Average,3.0,3.0,2020-11-01,685000.0,13.435637,63.0,POINT (-117.99081 33.89996),La Mirada
2,34.219352,-118.205306,2075.0,16153.0,LFR120,0.0,Very Low,3.0,Excellent,3.0,2.0,2020-11-01,1250000.0,14.037117,65.0,POINT (-118.20531 34.21935),La Canada Flintridge
3,33.894516,-118.245489,912.0,5241.0,CORL,22.0,High,6.0,Average,2.0,1.0,2020-11-01,450000.0,13.015466,109.0,POINT (-118.24549 33.89452),Compton
4,34.162450,-117.901663,2580.0,6360.0,AZRA,2.0,Moderate,3.0,Average,4.0,3.0,2020-11-01,800000.0,13.590830,17.0,POINT (-117.90166 34.16245),Azusa


In [47]:
print(f'There are  {gdf.neighborhood.isnull().sum()} out of {gdf.shape[0]} missing classifications.')
gdf.drop('geometry', axis=1, inplace=True)
gdf['neighborhood'].fillna('Missing', inplace=True)
gdf.to_csv('Data/houses_neighborhood_info.csv', index=False)
gdf.head()

There are  555 out of 5947 missing classifications.


,latitude,longitude,home_size,lot_size,zoning,sex_offenders,crime_index,enviornmental_hazards,school_quality,bedrooms,bathrooms,date,sale_price,sale_price_cpi,age,neighborhood
0,34.190532,-118.145081,1199.0,7207.0,LCR175,5.0,Moderate,4.0,Poor,3.0,2.0,2020-11-01,1010000.0,13.823924,70.0,Altadena
1,33.899958,-117.990815,1254.0,6033.0,LMR1,2.0,Low,3.0,Above Average,3.0,3.0,2020-11-01,685000.0,13.435637,63.0,La Mirada
2,34.219352,-118.205306,2075.0,16153.0,LFR120,0.0,Very Low,3.0,Excellent,3.0,2.0,2020-11-01,1250000.0,14.037117,65.0,La Canada Flintridge
3,33.894516,-118.245489,912.0,5241.0,CORL,22.0,High,6.0,Average,2.0,1.0,2020-11-01,450000.0,13.015466,109.0,Compton
4,34.162450,-117.901663,2580.0,6360.0,AZRA,2.0,Moderate,3.0,Average,4.0,3.0,2020-11-01,800000.0,13.590830,17.0,Azusa


### Create a DataFrame for the Basic Search component in the app

In [48]:
#Create a dataframe that will be used by the app
#Sale Price per neighborhood

avg = gdf.groupby('neighborhood')[['home_size', 'lot_size', 'bedrooms', 'bathrooms', 'sex_offenders', 
                                   'enviornmental_hazards', 'age', 'sale_price']].mean()

cat_avg = gdf.groupby('neighborhood')[['school_quality', 'crime_index']].agg(lambda x:x.value_counts().index[0])
avg['sale_price'] = avg['sale_price'].apply(lambda x: '${:,.2f}'.format(x))
new_avg = pd.merge(avg, hoods[['neighborhood', 'latitude', 'longitude']], on='neighborhood', how='left')
new_avg = pd.merge(new_avg, cat_avg, on='neighborhood', how='left')
new_avg = new_avg.dropna()

ohe_encoder = OneHotCategoricalEncoder(variables=['crime_index', 'school_quality'])
ohe_encoder.fit(new_avg)
new_avg = ohe_encoder.transform(new_avg)
if 'crime_index_Very High' not in new_avg.columns:
    new_avg['crime_index_Very High'] = 0
new_avg.to_csv('Data/Neighborhoods_final.csv', index=False)